In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('rent_reduced.csv')
# Drop zeros
df = df[df['usd price'] != 0]
df = df.dropna()
df.head()

,id,date,property_type,lat,lon,local price,usd price,barrio
0,9b254ef5797b842cf18729c4df9827332fdca285,8/26/2014,apartment,-34.607902,-58.390337,13757.06,796.38,43
1,6052eff893a6541b80eff7c92e2a84244a4a52f7,1/22/2015,apartment,-34.581524,-58.433261,12092.15,700.00,0
2,0127fd9d7382e8e8e98654b0f74c09a82d54f049,12/2/2014,apartment,-34.596975,-58.408897,10317.88,597.29,44
3,c7fc7d25b11f8cb7fa5ca9d8ea913422f99cde64,7/14/2014,apartment,-34.558106,-58.455842,21782.28,1260.95,1
4,356ec84cc967b6c2f26fabd548dfb2534ad7dd6a,8/24/2014,apartment,-34.542730,-58.462001,34393.01,1990.97,2


In [3]:
df.describe()

,lat,lon,local price,usd price,barrio
count,62503.000000,62503.000000,6.250300e+04,62503.000000,62503.000000
mean,-34.596041,-58.418921,3.961624e+04,2328.692932,21.771403
std,0.023843,0.040136,7.409684e+04,4324.020449,18.677357
min,-34.702011,-58.531322,5.298300e+02,33.180000,0.000000
25%,-34.610500,-58.446428,9.052700e+03,532.790000,1.000000
50%,-34.595852,-58.414660,1.834292e+04,1090.800000,19.000000
75%,-34.580811,-58.384781,4.425765e+04,2567.550000,43.000000
max,-34.534435,-58.354052,1.963500e+06,100000.000000,48.000000


In [4]:
df.date = pd.to_datetime(df.date)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62503 entries, 0 to 65946
Data columns (total 8 columns):
id               62503 non-null object
date             62503 non-null datetime64[ns]
property_type    62503 non-null object
lat              62503 non-null float64
lon              62503 non-null float64
local price      62503 non-null float64
usd price        62503 non-null float64
barrio           62503 non-null int64
dtypes: datetime64[ns](1), float64(4), int64(1), object(2)
memory usage: 4.3+ MB


In [5]:
lower_class = np.percentile(df['usd price'], 10)

In [6]:
upper_class = np.percentile(df['usd price'], 90)

In [7]:
r_df = df[['barrio', 'date', 'local price', 'usd price']]

aggregation = {
    'usd price': {
        'count': 'count',
        'mean': 'mean',
        'count 10th percentile': lambda x: (x <= lower_class).sum(),
        'count 90th percentile': lambda x: (x >= upper_class).sum(),
        'min': 'min',
        'max': 'max',
        'std': 'std'
    }
}

stats = r_df.groupby(['barrio']).agg(aggregation)

/home/rocket/miniconda3/envs/casas/lib/python3.7/site-packages/pandas/core/groupby/generic.py:1315: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [8]:
stats = pd.DataFrame(stats['usd price'][['count', 'mean', 'count 10th percentile', 'count 90th percentile', 'min', 'max', 'std']])

In [9]:
stats.head()

,count,mean,count 10th percentile,count 90th percentile,min,max,std
barrio,,,,,,,
0,11671,2474.764433,398.0,1305.0,51.09,94000.00,3301.069616
1,4427,2561.538692,240.0,744.0,33.18,40000.00,3225.375682
2,1607,2068.328681,68.0,222.0,116.66,26280.36,2650.870091
3,365,1602.897534,30.0,34.0,272.87,12100.00,2077.486876
4,2764,2188.151020,434.0,214.0,76.48,80000.00,4647.870154


In [10]:
stats.to_csv('barrio_stats.csv')

In [11]:
stats = r_df.groupby(['barrio', pd.Grouper(key="date", freq='M')]).agg(aggregation)

In [12]:
stats = pd.DataFrame(stats['usd price'][['count', 'mean', 'count 10th percentile', 'count 90th percentile', 'min', 'max', 'std']])

In [13]:
stats.head()

count         mean  count 10th percentile  \
barrio date                                                    
0      2014-07-31      3  1614.370000                    0.0   
       2014-08-31     18  2248.242222                    0.0   
       2014-09-30     33  2912.592424                    0.0   
       2014-10-31     22  1629.619091                    0.0   
       2014-11-30     56  2065.284286                    0.0   

                   count 90th percentile      min       max          std  
barrio date                                                               
0      2014-07-31                    0.0  1500.00   1743.11   122.190387  
       2014-08-31                    2.0   431.37  11000.00  3226.363649  
       2014-09-30                    7.0   479.36  10995.17  3020.857769  
       2014-10-31                    0.0   504.67   2767.13   933.460158  
       2014-11-30                    0.0   588.30   4762.47  1332.639077

In [14]:
stats.to_csv('barrio_month_stats.csv')